## Bi Encoder Preprocessing
- Grab sentence and article embeddings using mpnet 

In [20]:
import pandas as pd
import spacy 
from tqdm.auto import tqdm 
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from datasets import Dataset

In [12]:
deviceNum = 1
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")


In [3]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")

In [4]:
df.columns

Index(['Unnamed: 0', 'url1_lang', 'url2_lang', 'pair_id', 'link1', 'link2',
       'ia_link1', 'ia_link2', 'Geography', 'Entities', 'Time', 'Narrative',
       'Overall', 'Style', 'Tone', 'id1', 'id2', 'ogText1', 'ogTitle1',
       'ogText2', 'ogTitle2', 'text1', 'title1', 'text2', 'title2'],
      dtype='object')

In [5]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')


In [6]:
#nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"])
#nlp.enable_pipe("senter")
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

def extractSents(inList): 
    separator = nlp.pipe(inList, n_process=8)
    textList = []
    for text in tqdm(separator): 
        sentList = [str(sent) for sent in text.sents if len(sent) > 3]
        textList.append(sentList)
    return textList

df["sentences1"] = extractSents(df["text1"])
df["sentences2"] = extractSents(df["text2"])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [8]:
class CustomSentListDataset(Dataset): 
    def __init__(self, inDf): 
        self.inDf = inDf 
        
    def __len__(self): 
        return len(self.inDf)
    
    def __getitem__(self, idx):
        return list(self.inDf.iloc[idx])

In [9]:

def getLoader(inDf, batchSize): 
    inDf["text1_tokenized"] = inDf["sentences1"].apply(lambda x: tokenizer.batch_encode_plus(x, max_length=300, padding="max_length", truncation=True, return_tensors="pt"))
    inDf["text2_tokenized"] = inDf["sentences2"].apply(lambda x: tokenizer.batch_encode_plus(x, max_length=300, padding="max_length", truncation=True, return_tensors="pt"))
    
    inDf["text1_input_ids"] = inDf["text1_tokenized"].apply(lambda x: x["input_ids"])
    inDf["text2_input_ids"] = inDf["text2_tokenized"].apply(lambda x: x["input_ids"])
    
    inDf["text1_attention_mask"] = inDf["text1_tokenized"].apply(lambda x: x["attention_mask"])
    inDf["text2_attention_mask"] = inDf["text2_tokenized"].apply(lambda x: x["attention_mask"])
    
    #inDf["ground_truth"] = inDf["ground_truth"].apply(lambda x: x))
    
    dataset = CustomSentListDataset(inDf[["text1_input_ids", "text1_attention_mask","text2_input_ids", "text2_attention_mask"]])
    
    # convert dataset features to PyTorch tensors
    #dataset.set_format(type='torch', columns=["text1_input_ids", "text1_attention_mask","text2_input_ids", "text2_attention_mask"])
    loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
    return loader, inDf

loader, moddedDf = getLoader(df, 1)


In [13]:
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2').to(device)

In [14]:
for batch in tqdm(loader): 
    inputs_ids_a = batch[0][0].to(device)
    attention_a = batch[1][0].to(device)
    inputs_ids_b = batch[2][0].to(device)
    attention_b = batch[3][0].to(device)
    
    encoded_a = model(inputs_ids_a, attention_a)
    encoded_b = model(inputs_ids_b, attention_b)
        

  0%|          | 0/4876 [00:00<?, ?it/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-14-3856803e2590>:8 in <module>                                             │
│                                                                                           │
│ /opt/anaconda/lib/python3.9/site-packages/torch/nn/modules/module.py:1131 in _call_impl   │
│                                                                                           │
│   1128 │   │   # this function, and just call forward.                                    │
│   1129 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1130 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1131 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1132 │   │   # Do not call functions when jit is used                                   │
│   1133 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1134 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /opt/anaconda/lib/python3.9/site-packages/transformers/models/mpnet/modeling_mpnet.py:554 │
│ in forward                                                                                │
│                                                                                           │
│    551 │   │                                                                              │
│    552 │   │   head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)   │
│    553 │   │   embedding_output = self.embeddings(input_ids=input_ids, position_ids=posit │
│ ❱  554 │   │   encoder_outputs = self.encoder(                                            │
│    555 │   │   │   embedding_output,                                                      │
│    556 │   │   │   attention_mask=extended_attention_mask,                                │
│    557 │   │   │   head_mask=head_mask,                                                   │
│                                                                                           │
│ /opt/anaconda/lib/python3.9/site-packages/torch/nn/modules/module.py:1131 in _call_impl   │
│                                                                                           │
│   1128 │   │   # this function, and just call forward.                                    │
│   1129 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1130 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1131 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1132 │   │   # Do not call functions when jit is used                                   │
│   1133 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1134 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /opt/anaconda/lib/python3.9/site-packages/transformers/models/mpnet/modeling_mpnet.py:341 │
│ in forward                                                                                │
│                                                                                           │
│    338 │   │   │   if output_hidden_states:                                               │
│    339 │   │   │   │   all_hidden_states = all_hidden_states + (hidden_states,)           │
│    340 │   │   │                                                                          │
│ ❱  341 │   │   │   layer_outputs = layer_module(                                          │
│    342 │   │   │   │   hidden_states,                                                     │
│    343 │   │   │   │   attention_mask,            

In [23]:
np.array(encoded_a)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ <ipython-input-23-c965f4ff4f11>:1 in <module>                                             │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'tuple' object is not callable

In [ ]:
#NOTE: reference this later 
testEncoding = model(moddedDf.loc[1, "text1_input_ids"], max_length = 200, attention_mask=moddedDf.loc[1, "text1_attention_mask"])[0]

In [ ]:
for i, row in tqdm(moddedDf.iterrows()): 
    sentEncodings1 = model(row["text1_input_ids"], max_length = 200, attention_mask=row["text1_attention_mask"])[0]

In [ ]:
def embedSents(): 
    encodedList = [model() for item in inList]
    return encodedList

In [ ]:
moddedDf.head(3)
leanDf = moddedDf

In [ ]:
#TODO: pool all of the encodings to get one per sentence. Put this into an extra column in df
#TODO: do the same for the title and the first part of each article 

In [ ]:
testEncoding.shape

In [ ]:
def encode(inList): 
    return []

In [ ]:
extracted1[0]